In [181]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
from re import findall as fa
import sqlite3
import pymorphy2

from nltk.tokenize import word_tokenize

In [246]:
text = """
Иногда я пишу о книгах, которые произвели на меня впечатление. Писать большие отзывы сейчас не хочется, поэтому в порядке перечисления.
"Атлант расправил плечи" - за последнее время понравилась больше всего наряду с Довлатовым (но насчет последнего сомнений и не было). Почему-то раньше я думал, что это что-то вроде "Финансиста" Драйзера. Так же, видимо, думают и люди, рисующие мемы "сын маминой подруги расправил плечи". А на самом деле книга об альтернативной вселенной, где в США наступил социализм. Очень рекомендую.
Что до "Финансиста" Драйзера, то он надолго отбил у меня желание читать этого автора. Не потому что мне не интересно читать про рынок - наоборот, про рынок интересно. Но всё остальное там скучно, особенно герои. Может быть, так и было задумано, но я это не люблю.
Дилогия об Остапе Бендере - начинаются обе книги весело, кончаются обе книги уныло. Не столько с точки зрения событий, сколько с точки зрения того, как трансформируется язык. Поэтому от них остается неприятное ощущение, хотя написаны они ярко, весело и интересно. Впрочем, не пойти на такую сделку вряд ли можно было в условиях, в которых работали авторы.
"Три мушкетера". Ну, не побоюсь этого слова, такое. Занятно, но не более того - я сейчас даже с трудом вспомнил об этой книжке. Главный интерес книжка представляет с исторической точки зрения. В том числе и потому, что является убедительным доказательством, что во Франции в 17м веке был интернет и портативные телепорты - ну или по крайней мере бесстыдная сценарная магия.
Отто Кариус, "Тигры в грязи". Язык этой книги совершенно ужасен, может быть, потому что её писал солдат. Но прочитать очень стоит, потому что мало что может быть так ценно, как новая точка зрения на нечто хорошо знакомое!
"""

In [227]:
def cleanse(s):
    rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
    return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))

In [170]:
allpos= ['PRED', 'None', 'PRTS', 'ADJF', 'INFN', 
         'PRTF', 'NOUN', 'ADVB', 'VERB', 'NPRO', 
         'NUMR', 'CONJ', 'ADJS', 'PRCL', 'PREP', 'COMP', 'INTJ']

In [ ]:
def extract_features(text, 
                     morph=pymorphy2.MorphAnalyzer(), 
                     pos_types=['ADJF', 'NOUN', 'ADVB', 'VERB', 'CONJ', 'PREP'],
                    ):
    
    #length in chars and words
    len_char = len(text)
    len_word = len(text.split())
    pun = fa('[\.+,!\?:-]',text)
    n_pun = len(pun)
    braсket_list = fa('[\(\)]',text)
    
    #POS & grammem
    def cleanse(s):
        rgxp = '[\`\)\(\|©~^<>/\'\"\«№#$&\*.,;=+?!\—_@:\]\[%\{\}\\n]'
        return re.sub(' +', ' ', re.sub(rgxp, ' ', s.lower()))
    
    def parse_text(text, morph=morph):
        tokens = word_tokenize(cleanse(text))
        return [morph.parse(t) for t in tokens]
    
    parsed_text = parse_text(text)
    pos_list = [str(p[0].tag.POS) for p in parsed_text]
    n_nouns = len([t for t in pos_list if t=='NOUN'])
    n_verbs = len([t for t in pos_list if t=='VERB'])
    anim_list = [str(p[0].tag.animacy) for p in parsed_text]
    pers_list = [str(p[0].tag.person) for p in parsed_text]
    tns_list = [str(p[0].tag.tense) for p in parsed_text]
    asp_list = [str(p[0].tag.aspect) for p in parsed_text]
        
    features = {
        #surface features
        'len_char': len_char, 
        'len_word': len_word,
        'm_len_word': len_char / len_word,
        #punctuation
        'p_pun': len(pun) / len_char,
        'p_dot': len([i for i in pun if i=='.'])/len(pun),
        'p_qm': len([i for i in pun if i=='?'])/len(pun),
        'p_excl': len([i for i in pun if i=='!'])/len(pun),
        'p_comma': len([i for i in pun if i==','])/len(pun),
        'p_brkt': len(braсket_list)/len_char,
        'p_brkt_up': len([i for i in braсket_list if i==')'])/len(braсket_list),
        #POS form
        'pos_form': ' '.join(pos_list),
        'pos_richness': len(set(pos_list)),
        #grammem features
        'p_anim': len([t for t in anim_list if t=='anim'])/n_nouns,
        'p_1per': len([t for t in pers_list if t=='1per'])/n_verbs,
        'p_3per': len([t for t in pers_list if t=='3per'])/n_verbs,
        'p_past': len([t for t in tns_list if t=='past'])/n_verbs,
        #'p_fut': len([t for t in tns_list if t=='futr'])/n_verbs,
        'p_pres': len([t for t in tns_list if t=='pres'])/n_verbs,
        'p_perf': len([t for t in asp_list if t=='perf'])/n_verbs,
    }
    
    for f in pos_types:
        features['p_'+f] = len([t for t in pos_list if t==f])/len(pos_list)
        
    
        
    return features

In [252]:
%%time
extract_features(text)

Wall time: 17.6 ms


{'len_char': 1740,
 'len_word': 277,
 'm_len_word': 6.281588447653429,
 'p_1per': 0.36666666666666664,
 'p_3per': 0.43333333333333335,
 'p_ADJF': 0.10108303249097472,
 'p_ADVB': 0.1263537906137184,
 'p_CONJ': 0.10830324909747292,
 'p_NOUN': 0.2490974729241877,
 'p_PREP': 0.10469314079422383,
 'p_VERB': 0.10830324909747292,
 'p_anim': 0.2463768115942029,
 'p_brkt': 0.0011494252873563218,
 'p_brkt_up': 0.5,
 'p_comma': 0.4915254237288136,
 'p_dot': 0.3728813559322034,
 'p_excl': 0.01694915254237288,
 'p_past': 0.5333333333333333,
 'p_perf': 0.43333333333333335,
 'p_pres': 0.5333333333333333,
 'p_pun': 0.0339080459770115,
 'p_qm': 0.0,
 'pos_form': 'ADVB NPRO VERB PREP NOUN ADJF VERB PREP NPRO NOUN INFN ADJF NOUN ADVB PRCL VERB ADVB PREP NOUN NOUN NOUN VERB NOUN None PREP ADJF NOUN VERB COMP ADVB ADVB PREP NOUN CONJ PREP ADJF NOUN CONJ PRCL VERB ADVB COMP NPRO VERB CONJ PRCL NPRO PRCL NOUN NOUN CONJ PRCL ADVB VERB CONJ NOUN PRTF NOUN NOUN ADJF NOUN VERB NOUN CONJ PREP ADJF NOUN NOUN PREP 